In [1]:
%reload_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import plotly 
import plotly.graph_objects as go

sys.path.insert(0, "./../../src/")
from data_loader import load_data

## Loading the data

In [2]:
seasons = load_data()

In [3]:
# tmp_df = pd.read_csv("./../../data/raw/2017-18_pbp.csv")
# display(tmp_df)
display(seasons)

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,...,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,season_name
0,0,0,PERIOD_BEGIN,0,20001116,NaN,12:00,1,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:11 PM,2000-01
1,1,0,JUMP_BALL,1,20001116,Jump Ball Grant vs. Mutombo: Tip to Hill,12:00,1,4.0,5,...,Tyrone Hill,PHI,Philadelphia,1.610613e+09,76ers,NaN,NaN,NaN,12:12 PM,2000-01
2,2,Jump Shot,FIELD_GOAL_MADE,2,20001116,NaN,11:45,1,5.0,0,...,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Iverson 21' Jump Shot (2 PTS),12:13 PM,2000-01
3,3,1,FOUL,3,20001116,NaN,11:28,1,5.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hill P.FOUL (P1.T1),12:13 PM,2000-01
4,4,Jump Shot,FIELD_GOAL_MADE,4,20001116,Mason 17' Jump Shot (2 PTS),11:19,1,4.0,0,...,NaN,NaN,NaN,NaN,NaN,2 - 2,TIE,NaN,12:13 PM,2000-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10389750,431,1,TIMEOUT,634,21801037,NaN,0:04,4,3.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cavaliers Timeout: Regular (Reg.6 Short 0),10:49 PM,2018-19
10389751,432,0,SUBSTITUTION,635,21801037,NaN,0:04,4,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB: Clarkson FOR Zizic,10:50 PM,2018-19
10389752,433,Fadeaway Jumper,FIELD_GOAL_MISSED,638,21801037,NaN,0:02,4,5.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Stauskas 24' 3PT Fadeaway Jumper,10:51 PM,2018-19
10389753,434,0,REBOUND,639,21801037,Kleber REBOUND (Off:5 Def:7),0:02,4,4.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:51 PM,2018-19


In [4]:
seasons_subset = seasons[seasons["season_name"] == "2018-19"]
display(seasons_subset.head())

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,...,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,season_name
9807287,0,0,PERIOD_BEGIN,2,21800794,NaN,12:00,1,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:11 PM,2018-19
9807288,1,0,JUMP_BALL,4,21800794,Jump Ball Ayton vs. Faried: Tip to Okobo,12:00,1,4.0,5,...,Elie Okobo,PHX,Phoenix,1.610613e+09,Suns,NaN,NaN,NaN,9:11 PM,2018-19
9807289,2,1,TURNOVER,7,21800794,Jackson Bad Pass Turnover (P1.T1),11:39,1,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harden STEAL (1 STL),9:11 PM,2018-19
9807290,3,Driving Layup,FIELD_GOAL_MADE,9,21800794,NaN,11:31,1,5.0,0,...,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Harden 3' Driving Layup (2 PTS),9:12 PM,2018-19
9807291,4,2,FOUL,10,21800794,Bridges S.FOUL (P1.T1) (E.Dalen),11:31,1,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:12 PM,2018-19


## Visualizations 
Below you find a vizualization for each of the features found in the data  set to 

In [8]:
# Histogram of event types
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=seasons_subset["EVENTMSGTYPE"])
)
fig.update_layout(
    title_text="Histogram of Event Types", 
    xaxis_title="Event type name",
    yaxis_title="Frequency"
)
fig.show()

In [ ]:
# Histogram of event action types
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=seasons_subset["EVENTMSGACTIONTYPE"])
)
fig.update_layout(
    title_text="Histogram of Event Action Types", 
    xaxis_title="Event action type name",
    yaxis_title="Frequency"
)
fig.show()

In [ ]:
# Number of events per game 
events_per_game_count = seasons_subset.groupby("GAME_ID").count()["EVENTNUM"]

# Histogram of event action types
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=events_per_game_count)
)
fig.update_layout(
    title_text="Histogram of events per game", 
    xaxis_title="Number of events",
    yaxis_title="Count"
)
fig.show()
print(events_per_game_count.median(), events_per_game_count.std())

## Finding the average game duration
As well as the shortest and longest games

In [ ]:
duration_of_each_game = seasons_subset.groupby("GAME_ID").count()["WCTIMESTRING"]	

In [ ]:
# match all valid dates, and create new dataframes 
correct_time_format_regex = "^\s?([0][0-9]|[1][0-2]|[0-9]):[0-5][0-9]\s?(?:AM|PM|am|pm)$"
correct_t_mask = seasons["WCTIMESTRING"].str.match(correct_time_format_regex)
correct_t_df = seasons[correct_t_mask]
wrong_t_df = seasons[~correct_t_mask]

print(f"The number of wrong entries in df {len(wrong_t_df['WCTIMESTRING'])}"\
      f", of those unique {len(wrong_t_df['WCTIMESTRING'].unique())}")
display(correct_t_df)

# convert times into 24h format
correct_t_df["WCTIMESTRING"] = pd.to_datetime(correct_t_df["WCTIMESTRING"], 
                                              format='%I:%M %p').dt.strftime('%H:%M')

In [ ]:
grouped_time_df = correct_t_df.sort_values("EVENTNUM").groupby(["GAME_ID"]).agg({'WCTIMESTRING': list})
grouped_time_df

In [ ]:
from datetime import timedelta
grouped_time_df["game_start_time"] = np.nan
grouped_time_df["game_end_time"] = np.nan
for index, row in grouped_time_df.iterrows():
    # very dumb handling of edge cases and wrong values
    until_midnight = []
    after_midnight = []
    for time_str in row["WCTIMESTRING"]:
        if time_str <= "23:59" and time_str >= "10:59":
            until_midnight.append(time_str)
        else:
            after_midnight.append(time_str)
    curr_row_arr = sorted(until_midnight) + sorted(after_midnight)
    start_time = pd.to_datetime(curr_row_arr[0], format='%H:%M')
    end_time = pd.to_datetime(curr_row_arr[-1], format='%H:%M')
    
    if start_time.hour > 10 and end_time.hour < 10:
        # add one day, if clock goes over midnight
        end_time = end_time + timedelta(hours=24)
    grouped_time_df.loc[index, "game_start_time"] = start_time
    grouped_time_df.loc[index, "game_end_time"] = end_time

grouped_time_df["game_duration"] = grouped_time_df["game_end_time"] - \
                                            grouped_time_df["game_start_time"]
# convert to minutes
grouped_time_df["game_duration"] = grouped_time_df["game_duration"].dt.total_seconds().div(60).astype(int)
display(grouped_time_df)

In [ ]:
# remove wrong durations, no game lasts that long
grouped_time_df = grouped_time_df[(grouped_time_df["game_duration"] < 300) & (grouped_time_df["game_duration"] > 0)]    
# Histogram of event action types
fig = go.Figure()
fig.add_trace(go.Histogram(x=grouped_time_df["game_duration"]))
fig.update_layout(
    title_text="Distribution of game durations", 
    xaxis_title="Game duration in minutes",
    yaxis_title="Count"
)
fig.show()

In [ ]:
# find wrong game durations 
# wrong_durations_df = grouped_time_df[grouped_time_df["game_duration"] <= 0]
wrong_durations_df = grouped_time_df[grouped_time_df["game_duration"] < 300]
display(wrong_durations_df)

In [ ]:
wrong_durations_df.iloc[-1]["WCTIMESTRING"]